## Random Projection

Scikit-learn 의 LSHForest 는 한 종류의 metric 만 이용 가능하며, LSH 의 문제점들을 개선하는 방향으로 추가적인 개발이 되지 않았습니다. Scikit-learn 의 0.21 이후부터는 LSHForest 는 더 이상 제공되지 않는다고 합니다. 또한 Scikit-learn 의 다른 인덱서인 BallTree 와 KDTree 는 sparse matrix 에 대한 인덱싱을 지원하지 않습니다.

안정적인 성능을 내기 위해서는 많은 작업들이 필요하지만, 기본적인 LSH 의 코드는 손쉽게 만들 수 있습니다. 이 튜토리얼을 통하여 Random Projection 을 이용하는 LSH 를 직접 개발해 봅니다.

Hash function 의 설계에 따라 다양하게 인덱서를 변형할 수 있습니다.

In [1]:
import numpy as np
from sklearn.preprocessing import normalize
from lovit_textmining_dataset.navernews_10days import get_bow

X, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')

In [2]:
X.shape

(30091, 9774)

numpy.random.random_samples 는 [0, 1) 사이의 값을 생성하기 때문에 0.5 를 뺍니다.

In [3]:
def generate_mapper(input_dim, output_dim, b_scale=0.1):
    # uniform distribution
    M = np.random.random_sample((input_dim, output_dim)) - 0.5
    M = normalize(M, axis=0, norm='l2')
    b = (np.random.random_sample(1) - 0.5) * b_scale
    return M, b

n_codes = 10
M, b = generate_mapper(X.shape[1], n_codes)
M.shape

(9774, 10)

In [4]:
b

array([0.01365153])

Sparse matrix 와 numpy.ndarray 의 곱은 safe_sparse_dot 함수를 이용해야 효율적입니다. safe_sparse_dot 함수는 sparse matrix 의 nonzero element 기준으로 곱셈을 수행합니다.

In [5]:
from sklearn.utils.extmath import safe_sparse_dot

def transform(X, mapper, dense_output=False):
    Y = safe_sparse_dot(X, mapper, dense_output)
    return Y

Y = transform(X, M)
Y.shape

(30091, 10)

numpy.ndarray 형식의 float vector 는 dtype 을 변경하여 손쉽게 integer vector 로 만들 수 있습니다.

In [6]:
Y[1] / 0.01

array([-2.24423257, -0.13922416,  0.39920841, -4.25557761, -2.9478034 ,
       -5.73837581, -1.00782216, -2.86794736, -0.70919218,  1.64046998])

In [7]:
np.asarray(Y[1] / 0.01, dtype=np.int)

array([-2,  0,  0, -4, -2, -5, -1, -2,  0,  1])

Sign 함수를 이용할 수도 있습니다.

In [8]:
np.sign(np.asarray(Y[1] / 0.01, dtype=np.int))

array([-1,  0,  0, -1, -1, -1, -1, -1,  0,  1])

## Locality Sensitive Hashing

앞서 만든 transform 함수를 이용하여 hash function 인 encode 함수를 만듭니다. Python 의 list 는 hashing 이 되지 않기 때문에 key 의 데이터 타입을 tuple 로 변경합니다. 모든 데이터에 대한 hash code 를 만든 뒤, 이를 기준으로 dict 에 row index 를 추가합니다.

In [9]:
from collections import defaultdict

def encode(X, mapper, b, radius=1):
    b_ = b / radius
    # Y = (transform(X, mapper) - b) / radius
    Y = np.sign((transform(X, mapper) - b))
    C = np.asarray(Y, dtype=np.int)
    # numpy.ndarray to tuple
    C = [tuple(c) for c in C.tolist()]
    return C

def indexing(C, bucket=None):
    if bucket is None:
        bucket = defaultdict(lambda: [])
    elif isinstance(bucket, dict):
        bucket = defaultdict(lambda: [], bucket)

    for i, c in enumerate(C):
        bucket[c].append(i)
    return dict(bucket)

In [10]:
radius = 0.1

C = encode(X, M, b, radius)
buckets = indexing(C)

단 한번이 random projection 으로는 buckets 의 사이즈가 불균형적입니다.

In [11]:
for code, bucket in buckets.items():
    if len(bucket) < 100:
        continue
    print('{} has {} items'.format(code, len(bucket)))

(-1, -1, -1, -1, -1, -1, -1, -1, -1, -1) has 1121 items
(-1, -1, -1, -1, -1, -1, -1, -1, -1, 1) has 197 items
(1, -1, -1, -1, -1, 1, -1, -1, -1, -1) has 126 items
(-1, -1, -1, -1, 1, 1, 1, -1, -1, -1) has 119 items
(-1, -1, -1, -1, -1, 1, 1, -1, -1, -1) has 113 items
(-1, -1, -1, 1, -1, -1, 1, -1, -1, 1) has 138 items
(1, -1, -1, -1, -1, 1, 1, -1, -1, -1) has 124 items
(-1, 1, -1, 1, -1, -1, -1, -1, 1, -1) has 101 items
(-1, -1, -1, -1, -1, -1, -1, 1, -1, -1) has 101 items
(1, -1, -1, -1, -1, -1, -1, -1, -1, -1) has 117 items
(1, 1, -1, -1, -1, -1, -1, -1, -1, 1) has 128 items
(1, -1, 1, -1, -1, -1, -1, -1, -1, -1) has 123 items
(1, -1, -1, 1, 1, 1, 1, -1, -1, -1) has 116 items
(-1, -1, -1, -1, -1, 1, -1, -1, -1, -1) has 133 items
(1, -1, -1, -1, 1, 1, 1, -1, -1, -1) has 230 items
(-1, -1, -1, -1, 1, -1, -1, -1, -1, -1) has 133 items
(-1, -1, -1, -1, 1, -1, 1, -1, -1, 1) has 150 items
(-1, -1, -1, 1, 1, 1, -1, -1, 1, -1) has 150 items
(1, -1, -1, 1, 1, 1, 1, -1, 1, 1) has 133 items
(-1

검색된 뉴스를 확인하기 위하여 뉴스를 list 로 읽어둡니다.

In [12]:
from lovit_textmining_dataset.navernews_10days import get_news_paths
from soynlp.utils import DoublespaceLineCorpus

path = get_news_paths(date='2016-10-20')
docs = list(DoublespaceLineCorpus(path))

인덱싱 과정은 앞서 구현한 함수를 이용하며, 검색 함수만 따로 구현하였습니다.

In [13]:
from sklearn.metrics import pairwise_distances

class HashingBasedIndexer:
    def __init__(self, n_codes=10, n_layers=10, radius=1, b_scale=0.05):
        self.n_codes = n_codes
        self.n_layers = n_layers
        self.radius = radius
        self.b_scale = b_scale

    def train(self, X):
        input_dim = X.shape[1]
        self.X = X
        self._generate_mapper(input_dim, self.n_layers)
        self._indexing(X, self.mappers, self.biases)

    def _generate_mapper(self, input_dim, n_layers):
        # initialize
        self.biases = []
        self.buckets = []
        self.mappers = []

        # generate mapper
        for _ in range(n_layers):
            M, b = generate_mapper(input_dim, n_codes)
            self.mappers.append(M)
            self.biases.append(b)

    def _indexing(self, X, mappers, biases):
        for M, b in zip(mappers, biases):
            C = encode(X, M, b, self.radius)
            self.buckets.append(indexing(C))

    def find_similar(self, query_vec, topk=10, candidates_factor=1.0, debug=False):
        candidates = self._get_candidates(query_vec, topk, candidates_factor)
        if debug:
            print('num candidatse = {}'.format(candidates.shape[0]))
        dist, idxs = self._compute_distance(query_vec, candidates, topk)
        return dist, idxs

    def _get_candidates(self, query_vec, min_num, candidates_factor):
        # {idx:co-occurrence}
        cooccurrences = defaultdict(int)
        for M, b, bucket in zip(self.mappers, self.biases, self.buckets):
            C = encode(query_vec, M, b, self.radius)[0]
            for idx in bucket.get(C, []):
                cooccurrences[idx] += 1

        # {co-occurrence:[idx, ...]}
        group_by = defaultdict(lambda: [])
        for idx, count in cooccurrences.items():
            group_by[count].append(idx)

        n_max_candidates = int(candidates_factor * min_num)
        candidates = []
        for count in sorted(group_by, key=lambda x:-x):
            if len(candidates) >= n_max_candidates:
                break
            candidates += group_by[count]

        # as numpy.ndarray
        return np.asarray(candidates, dtype=np.int)

    def _compute_distance(self, query_vec, candidates, topk):
        dist = pairwise_distances(self.X[candidates], query_vec, metric='cosine').reshape(-1)        
        sim_ref_idxs = dist.argsort()[:topk]
        sim_idxs = candidates[sim_ref_idxs]
        sim_dist = dist[sim_ref_idxs]
        return sim_dist, sim_idxs

구현한 함수를 테스트해 봅니다.

In [14]:
%%time
indexer = HashingBasedIndexer(n_codes=6, n_layers=8, radius=0.3)
indexer.train(X)

CPU times: user 636 ms, sys: 64 ms, total: 700 ms
Wall time: 698 ms


Bucket 만 잘 나뉘어진다면 실제 거리 계산을 수행하는 횟수가 크게 줄어듭니다. 그리고 그 성능은 hash function 의 설계에 전적으로 달려있습니다.

In [15]:
query = 15
query_vec = X[query].reshape(1,-1)

In [16]:
%%time

dist, idxs = indexer.find_similar(query_vec, candidates_factor=5, debug=True)

num candidatse = 460
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 7.43 ms


In [17]:
for d, idx in zip(dist, idxs):
    print('[idx = {}, cos = {:.3}] {} ...\n'.format(idx, 1 - d, docs[idx][:200]))

[idx = 15, cos = 1.0] 클린턴 득표율 50 목표 가능성 아주 크다  워싱턴 연합뉴스 강영두 특파원 미국 민주당 대선후보 힐러리 클린턴이 다음달 8일 대선에서 400명 이상의 선거인단을 확보하면서 대승을 거둘 수 있다는 분석이 나왔다  미국 민주당 대선후보 힐러리 클린턴이 지난 10일 현지시간 오하이오 주 콜럼버스에서 유세하는 모습 연합뉴스  미 공화당 전략가인 스티브 슈미트는 1 ...

[idx = 7421, cos = 0.74] 여론조사서는 304명 확보 예상  힐러리 클린턴 미국 민주당 대선후보가 다음달 8일 대선에서 과반은 물론 선거인단의 4분의 3 이상 확보할 것이라는 분석이 공화당에서 나왔다  미 공화당 전략가인 스티브 슈미트는 19일 현지시간 방송과의 인터뷰에서 이같이 전망했다 그는 지난 2008년 대선 당시 공화당 후보였던 존 메케인 캠프에 관여한 인물이다  그는 현 상 ...

[idx = 13752, cos = 0.632] 미 대선 3주 앞 판세 분석  미국 대선을 3주일 앞두고 치러진 19일 현지시간 의 마지막 대선후보 토론을 가리켜 정치전문매체 폴리티코 등은 공화당 도널드 트럼프 후보의 생사의 순간 이라 불렀다 트럼프는 앞선 두 번의 토론에 비해 민주당 힐러리 클린턴 후보에 대한 인신공격을 자제하고 차분하게 말하려 작심한 기색이 역력했다 하지만 판세를 뒤집기에는 역부족이었 ...

[idx = 7563, cos = 0.57] 공화당 텃밭서도 우세 유타 주는 맥멀린 1위 이변  19일 현지시간 미국 라스베이거스 네바다주에서 3차 토론을 벌이고 있는 힐러리 클린턴 오른쪽 민주당 후보와 도널드 트럼프 공화당 후보 라스베이거스 연합뉴스  11월 8일 치러질 미국 대통령 선거를 3주 앞두고 민주당 힐러리 클린턴 후보의 지지율이 주요 경합주에서 도널드 트럼프 공화당 후보를 앞서고 있다   ...

[idx = 558, cos = 0.567] 클린턴 공화당 텃밭서도 우세 유타 맥멀린 1위 이변  현재 지지율 호감도 1월 수준으로 회귀 

## Various Hash Function (NMF based Hash Function)

NMF 로 학습한 components 를 mapper 로 이용할 수도 있습니다. Sign 함수와 함께 이용하면 각 component 에 해당하는 토픽과 비슷하면 1 의 hash code value 를 가지게 됩니다.

In [18]:
import pickle
with open('../day05_topicmodeling/2016-10-20-nmf.pkl', 'rb') as f:
    nmf_model = pickle.load(f)
components = nmf_model.components_
components.shape

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.nmf module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator NMF from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


(100, 9774)

Permutation 을 이용하여 components 를 sampling 합니다.

In [19]:
print(np.random.permutation(10))

def sample_index(high, n_samples):
    return np.random.permutation(high)[:n_samples]

[5 1 8 3 9 0 6 2 7 4]


mapper 와 bias 를 따로 만든 뒤, 다시 한 번 인덱싱을 합니다.

In [20]:
components = normalize(components, norm='l2')
n_layers = 16
n_codes = 5
n_components = components.shape[0]

custom_mapper = [
    components[sample_index(n_components, n_codes)].transpose() 
    for _ in range(n_layers)
]

custom_bias = [
    np.random.random_sample(1) * 0.05
    for _ in range(n_layers)
]

In [21]:
indexer.radius = 0.5
indexer._indexing(X, custom_mapper, custom_bias)

NMF 로 만든 인덱서도 테스트해봅니다.

In [22]:
%%time

dist, idxs = indexer.find_similar(query_vec, candidates_factor=2, debug=True)

num candidatse = 460
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.24 ms


In [23]:
for d, idx in zip(dist, idxs):
    print('[idx = {}, cos = {:.3}] {} ...\n'.format(idx, 1 - d, docs[idx][:200]))

[idx = 15, cos = 1.0] 클린턴 득표율 50 목표 가능성 아주 크다  워싱턴 연합뉴스 강영두 특파원 미국 민주당 대선후보 힐러리 클린턴이 다음달 8일 대선에서 400명 이상의 선거인단을 확보하면서 대승을 거둘 수 있다는 분석이 나왔다  미국 민주당 대선후보 힐러리 클린턴이 지난 10일 현지시간 오하이오 주 콜럼버스에서 유세하는 모습 연합뉴스  미 공화당 전략가인 스티브 슈미트는 1 ...

[idx = 7421, cos = 0.74] 여론조사서는 304명 확보 예상  힐러리 클린턴 미국 민주당 대선후보가 다음달 8일 대선에서 과반은 물론 선거인단의 4분의 3 이상 확보할 것이라는 분석이 공화당에서 나왔다  미 공화당 전략가인 스티브 슈미트는 19일 현지시간 방송과의 인터뷰에서 이같이 전망했다 그는 지난 2008년 대선 당시 공화당 후보였던 존 메케인 캠프에 관여한 인물이다  그는 현 상 ...

[idx = 13752, cos = 0.632] 미 대선 3주 앞 판세 분석  미국 대선을 3주일 앞두고 치러진 19일 현지시간 의 마지막 대선후보 토론을 가리켜 정치전문매체 폴리티코 등은 공화당 도널드 트럼프 후보의 생사의 순간 이라 불렀다 트럼프는 앞선 두 번의 토론에 비해 민주당 힐러리 클린턴 후보에 대한 인신공격을 자제하고 차분하게 말하려 작심한 기색이 역력했다 하지만 판세를 뒤집기에는 역부족이었 ...

[idx = 7563, cos = 0.57] 공화당 텃밭서도 우세 유타 주는 맥멀린 1위 이변  19일 현지시간 미국 라스베이거스 네바다주에서 3차 토론을 벌이고 있는 힐러리 클린턴 오른쪽 민주당 후보와 도널드 트럼프 공화당 후보 라스베이거스 연합뉴스  11월 8일 치러질 미국 대통령 선거를 3주 앞두고 민주당 힐러리 클린턴 후보의 지지율이 주요 경합주에서 도널드 트럼프 공화당 후보를 앞서고 있다   ...

[idx = 558, cos = 0.567] 클린턴 공화당 텃밭서도 우세 유타 맥멀린 1위 이변  현재 지지율 호감도 1월 수준으로 회귀 